In [ ]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [1]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
else:
    !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'RI',
    root_path = root_path,
)

year = 2018
# self.get_shapes()#overwrite=True)
# self.get_pl()#overwrite=True)
# self.get_assignments()#overwrite=True)
# self.get_geo()#overwrite=True)
# self.get_crosswalks()#overwrite=True)
self.get_transformer(overwrite=True)

# self.get_acs5(year=year, overwrite=True)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
crosswalks.RI_block2020 ... Elapsed time: 0.1318 seconds
crosswalks.RI_block2020 ... Elapsed time: 0.1515 seconds
transformer.RI_tract2018 building ... 
select
    A.tract2010,
    A.block2020,
    A.block_group2020,
    A.tract2020,
    A.county2020,
    A.state2020,
    A.vtd2020,
    case when B.all_tot_pop = 0 then 0 else A.all_tot_pop / B.all_tot_pop end as all_tot_pop,
    case when B.all_vap_pop = 0 then 0 else A.all_vap_pop / B.all_vap_pop end as all_vap_pop,
    case when B.white_tot_pop = 0 then 0 else A.white_tot_pop / B.white_tot_pop end as white_tot_pop,
    case when B.white_vap_pop = 0 then 0 else A.white_vap_pop / B.white_vap_pop end as white_vap_pop,
    case when B.hisp_tot_pop = 0 then 0 else A.hisp_tot_pop / B.hisp_tot_pop end as hisp_tot_pop,
    case when B.hisp_vap_pop = 0 then 0 else A.hisp_vap_pop / B.hisp_vap_pop end as hisp_vap_po

'transformer.RI_tract2018'

In [3]:
ut.prep(ut.listify(['h', None]))

['h', None]

In [2]:
subpops = {
    'all_tot_pop'  : 'p2_001n',
    'all_vap_pop'  : 'p4_001n',
    'wite_tot_pop' : 'p2_005n',
    'white_vap_pop': 'p4_005n',
    'hisp_tot_pop' : 'p2_002n',
    'hisp_vap_pop' : 'p4_002n',
    'other_tot_pop': None,
    'other_vap_pop': None,
}
{v:k for k,v in subpops.items()}
# dict(zip(d.items()))

{'p2_001n': 'all_tot_pop',
 'p4_001n': 'all_vap_pop',
 'p2_005n': 'wite_tot_pop',
 'p4_005n': 'white_vap_pop',
 'p2_002n': 'hisp_tot_pop',
 'p4_002n': 'hisp_vap_pop',
 None: 'other_vap_pop'}